# Inverse kinematics
This notebook considers the problem of inverse kinematics, ie solving at each control cycle of the robot a quadratic program from the derivatives (jacobian) of the current state. It introduces the basic function to compute the Jacobians of the robot, and how to use them to compute an inverse-kinematics control law. One of the key difficulties is to understand in which frames each quantities is computed (might be in the world frame, in the local frame attached to the end effector, in some arbitrary goal frame, etc), as we should never mixed quantities expressed in different frames without transporting them in the right frame. 


In [26]:
import magic_donotload

## Set up
We will use a Gepetto-viewer, and the linear algebra of NumPy.

In [ ]:
# %load tp3/generated/inverse_kinematics_import
import pinocchio as pin
import numpy as np
import time
from numpy.linalg import pinv,inv,norm,svd,eig
from tiago_loader import loadTiago
import matplotlib.pylab as plt; plt.ion()
from utils.meshcat_viewer_wrapper import MeshcatVisualizer


We will use the Tiago robot (https://youtu.be/6BwRqwD066g). This mobile manipulator from PAL Robotics as a mobile basis which can move in the plane (3 dof), a manipulator arm (7 dof) and a head (2 dof) both mounted on a prismatic axis moving vertically (1 dof). This makes 3 dof for the basis, and 9 dof for the body. It also has 2 extra joints tot figure the wheels, which are not very useful for this notebook. The wheels and the basis rotations are represented by the cos and sin of the angle. The size of the configuration vector is then 18, while the velocity vector has dimension 15.

A load function is available to make it easy to load the robot.

In [ ]:
# %load tp3/generated/inverse_kinematics_robot
robot = loadTiago()
viz = MeshcatVisualizer(robot,url='classical')


In [30]:
viz.viewer.jupyter_cell()

The configuration is represented by a vector of larger dimension, subject to constraints (cos^2+sin^2=1). It is not possible to randomly sample a configuration vector q, as these constraints should be respected. Similarly, we should take care when integrating a velocity as summing a configuration q with a velocity v will not work (dimensions do not match). Two functions in Pinocchio implements these functionnalities.

In [31]:
q = pin.randomConfiguration(robot.model)
vq = np.random.rand(robot.model.nv)*2-1
DT = 1e-2
qnext = pin.integrate(robot.model,q,vq*DT)

A simple example moving the robot in the viewer following a constant (random) velocity is as follows.

In [32]:
for t in range(1000):
    q = pin.integrate(robot.model,q,vq*DT)
    viz.display(q)
    time.sleep(DT/10)

The robot is mobile, hence the camera view in the viewer is not always centered. 

## Forward kinematics and Jacobian
We recall first the basic method to compute the robot forward kinematics.
We will consider two frames of interest on the robot: the first one, named <tool> is at the tip of the end-effector ; the second one, named <basis>, is on the front of the robot basis, 10 cm above the ground. Both are represented in Gepetto-Viewer by a frame composed of three RGB arrows. 

In [10]:
%load tp3/generated/inverse_kinematics_frames

Frame name: frametool paired to (parent joint/ previous frame)(8/52)
with relative placement wrt parent joint:
  R =
1 0 0
0 1 0
0 0 1
  p =    0    0 0.08
containing inertia:
  m = 0
  c = 0 0 0
  I = 
0 0 0
0 0 0
0 0 0

In [34]:
robot.model.frames[IDX_TOOL]

Frame name: frametool paired to (parent joint/ previous frame)(8/52)
with relative placement wrt parent joint:
  R =
1 0 0
0 1 0
0 0 1
  p =    0    0 0.08
containing inertia:
  m = 0
  c = 0 0 0
  I = 
0 0 0
0 0 0
0 0 0

### Computing frame placement
These frames are computed by the Pinocchio function framesForwardKinematics, whose results are stored in robot.data.oMf.     

In [33]:
pin.framesForwardKinematics(robot.model,robot.data,q)
print("Tool placement:",robot.data.oMf[IDX_TOOL])
print("Basis placement:",robot.data.oMf[IDX_BASIS])

Tool placement:   R =
 0.809241 -0.514412 -0.283741
 0.330757  0.798108  -0.50361
 0.485519  0.313692  0.816007
  p = 0.0646125 0.0949587  -7.59329

Basis placement:   R =
1 0 0
0 1 0
0 0 1
  p =  0.3    0 0.15



It is very important to notice the instruction pattern (which is standard in all Pinocchion functions): first call a whole-body algorithm (here *pin.framesForwardKinematics*), then access the results into *robot.data*. 

The tool-placement matrix oMtool represents the displacement from the world frame F_o to the tool frame F_tool. It is composed on a rotation matrix oRtool and a 3D vector o_OT: oMf = [ oRf o_OT ], when o_OT is the vector from the origin of frame F_o to the origin of from F_tool ; this vector is expressed in the world frame F_o. We can rather express OT in the tool frame F_tool by multiplying it by oRtool:

In [35]:
oMtool = robot.data.oMf[IDX_TOOL]
oRtool = oMtool.rotation; o_OT = oMtool.translation
tool_OT = oRtool.T @ o_OT

Take care to the multiplication operator in numpy. The operator* is (unintuitively) mapped to the coefficient-wise multiplication ... i.e. not at all the matrix multiplication. You should use the operator @ to get the real matrix-matrix product. 
Never mind, you will likely be tricked at least once by this design pattern. If you don't like it ... well there is nothing to do, this is the basic matrix library in Python, the world most used language. 

### Computing Jacobians
The jacobian of a frame of the robot is computed using pio.computeFrameJacobian.

In [36]:
Jtool = pin.computeFrameJacobian(robot.model,robot.data,q,IDX_TOOL)
Jtool.shape

(6, 12)

This matrix has 6 rows and NV=15 columns. It corresponds to the "spatial" 6D velocity of the end effector. Let's first focus on the 3 first rows, corresponding to the linear velocity.

In [38]:
Jtool3 = Jtool[:3,:]

#### Jacobian as a velocity operator
A first way to understand what is this matrix is to see that as an operator that converts the velocity in the configuration space into the linear velocity of the end effector.

In [39]:
vtool = Jtool3 @ vq

But in which frame is vtool expressed? The choice in Pinocchio (following algorithmic principles described in [Featherstone 2009]) is to expressed quantities in the local frame by default. So vtool is expressed in the tool frame F_tool (or more precisely, the Galilean frame coinciding with F_tool at the current time instant). We should better denote it with its frame.

In [40]:
tool_Jtool = pin.computeFrameJacobian(robot.model,robot.data,q,IDX_TOOL)
tool_Jtool3 = tool_Jtool[:3,:]
tool_vtool = tool_Jtool3 @ vq

We are using here long variable names. In the code, these explicit notations are maybe to much. It is your choice to use them or not. For sure, you should use similar notations on the paper board when formulating your algorithm, but maybe you prefer lighter variable naming for implementing it.
The tool velocity is easier to interpret in the world frame.

In [41]:
o_vtool = oRtool @ tool_vtool

We generalize this notation to the jacobian expressed in the world frame F_o:

In [42]:
o_Jtool3 = oRtool @ tool_Jtool3

#### Jacobian as a derivative
A second interpretation to explain what is the jacobian is to observe that it is the derivative of the vector o_OT (the tool position in world frame). Indeed o_OT is a function of q: o_OT(q). We can take its derivative with respect to q, denoted d o_OT / dq. This derivatives is equal to the Jacobian expressed in the world frame F_o: d o_OT / dq = o_Jtool.
To be convinced of that, let's check the finite differences. We take a small movement dq, and see that the change in position o_OT(q+dq) corresponds to the linear prediction o_Jtool3*dq.

In [43]:
# Sample between -0.001 and 0.001
EPS = 1e-3
dq = (np.random.rand(robot.model.nv)*2-1)*EPS

# q2 = q+dq
q2 = pin.integrate(robot.model,q,dq)                     
pin.framesForwardKinematics(robot.model,robot.data,q)

# tool position for q
o_OT = robot.data.oMf[IDX_TOOL].translation.copy()

# tool position for q+dq
pin.framesForwardKinematics(robot.model,robot.data,q2)
o_OT2 = robot.data.oMf[IDX_TOOL].translation.copy()

print((o_OT2 - o_OT).T/EPS)
print((o_Jtool3@dq/EPS).T)

[-0.06400179 -0.07806081 -0.46450085]
[-0.06406556 -0.078047   -0.46448161]


#### Frame options in Pinocchio
Most algorithms accept an option to specify in wich frame the spatial quantity should be expressed. The two basic options are *pin.LOCAL* and *pin.WORLD*. When related to velocity, *LOCAL* is the linear velocity of the center of the local frame (the TOOL_IDX frame, here) and the angular velocity, both expressed in the local frame. With *WORLD* frame, the two velocities are expressed in the world frame ... but remember that the linear velocity is then difficult to interpret.

A last option is given by convenience, which does not respect the mathematics of spatial velocity, but matches the French "torseur cinématique": pin.LOCAL_WORLD_ALIGNED gives the linear velocity of the center of the local frame and the angular velocity, both expressed in the world frame. It is convenient especially when we are interested to consider the linear velocity as the derivative of the position.

In [44]:
o_Jtool = pin.computeFrameJacobian(robot.model,robot.data,q,IDX_TOOL,
                                   pin.LOCAL_WORLD_ALIGNED)[:3,:]

You may see this video which explain with more illustriations the 3 frame options:  https://youtu.be/MLFtHLTprE4 

#### Jacobian in 6D
The jacobian of the frame indeed as 6 rows, and corresponds to the spatial velocity of the frame. It is expressed locally, in the tool frame (or more precisely the Galilean frame coincinding with the tool frame to the current time instant).

In [46]:
tool_J = pin.computeFrameJacobian(robot.model,robot.data,q,IDX_TOOL)
tool_nu = tool_J @ vq

We denote by nu the spatial velocity. It is composed by the linear velocity of the center of the frame vtool and the angular velocity of the tool frame, both expressed in the tool frame: tool_nu = [ tool_vtool, tool_w ].



In [47]:
tool_vtool = tool_nu[:3] ; tool_w = tool_nu[3:]

We can rather expressed the spatial velocity in the world frame F_o by multiplying it by the so-called action matrix oXtool.

In [48]:
o_nu = oMtool.action @ tool_nu
print(o_nu)

[-10.27701894   1.50315172  -1.07705109  -0.16997645  -1.35064114
  -0.94447856]


#### Log in SE(3)
Finally, the log operator transforms a displacement M in SE(3) into the spatial (6D) velocity that should be applied during 1 second to achieve this displacement. The spatial velocity is expressed in the frame at the origin of the displacement (i.e. nu=log(oMtool) is expressed in the world frame F_o). In Pinocchio, the log returns an object of class Motion, that can be converted to a numpy vector.

In [49]:
o_nu = pin.log(oMtool).vector
print(o_nu.T)

[-1.6717141  -1.51713667 -7.3815157   0.45309705 -0.42646367  0.46854601]


## Inverse kinematics for the moving the robot effector
We will first move only the robot end effector, to reach a target defined by a frame F_goal.

In [ ]:
%load -r 22-28 tp3/generated/inverse_kinematics_goal

### Position the effector (3d)
It is time to write your first control law. Write a for-loop to iterate along the control cycles of the robot. At each control cycle, you should:
* compute the Jacobian 3D in the world frame o_Jtool3
* compute the vector from the tool to the goal, expressed in world frame: o_TG = o_goal - o_tool
* compute the control law as vq=-pinv(J)*o_TG
* integrated the velocity vq during DT to get a new configuration q.
You might want to start from the following initial configuration, or from any random one.

In [ ]:
%load tp3/generated/inverse_kinematics_init


It is a good idea to store the values of the error between tool and goal o_TG, to plot them later. For that, simply append each o_TG computed at every control cycle in a list.

In [ ]:
# %load tp3/generated_inverse_kinematics_3d_loop
q = q0.copy()
herr = [] # Log the value of the error between tool and goal.
# Loop on an inverse kinematics for 200 iterations.
for i in range(200):  # Integrate over 2 second of robot life

    # Run the algorithms that outputs values in robot.data
    pin.framesForwardKinematics(robot.model,robot.data,q)
    pin.computeJointJacobians(robot.model,robot.data,q)

    # Placement from world frame o to frame f oMtool
    oMtool = robot.data.oMf[IDX_TOOL]

    # 3D jacobian in world frame
    o_Jtool3 = pin.computeFrameJacobian(robot.model,robot.data,q,IDX_TOOL,pin.LOCAL_WORLD_ALIGNED)[:3,:]

    # vector from tool to goal, in world frame
    o_TG = oMtool.translation-oMgoal.translation
    
    # Control law by least square
    vq = -pinv(o_Jtool3)@o_TG

    q = pin.integrate(robot.model,q, vq * DT)
    viz.display(q)
    time.sleep(1e-3)

    herr.append(o_TG) 


It is interesting to plot the behavior of the robot. If the error at each iteration has been stored as a list of 3x1 matrices, the following code plots it.

In [ ]:
plt.plot(herr)
plt.xlabel('control cycle (iter)')
plt.ylabel('error (m)');

We can observe that each component of the error converges toward 0 following an exponential trajetory. The convergence is assymptotic. To fasten the convergence, increase the gain of the control law ($v_q = - \lambda J^+ e$), where the gain $\lambda$ has yet been set to 1.

### Place the end effector (6D)
The previous control law brings the center of the effector toward the center of the goal frame. However, it does not control the orientation of the end effector: the axes of the two frames F_tool and F_goal do not converge.
We should now modify the control law to take into account the tool orientation. For that, we compute the error to be the SE(3) log of the displacement from the tool frame F_tool to the goal frame F_goal.

In [ ]:
toolMgoal = oMtool.inverse()*oMgoal
tool_nu = pin.log(toolMgoal).vector

This error tool_nu is a 6d vector, that can be interpreted as the spatial (6d) velocity that should be applied during 1 second to displace the tool frame F_tool (placed at oMtool) to the goal frame F_goal (placed at oMgoal). This spatial velocity is expressed in the tool frame F_tool. It then corresponds to the 6D jacobian, that is also computed in the same frame F_tool.

Implement a second control law, following the same pattern than the previous control law. At each control cycle, you should:
* compute the displacement between F_tool and F_goal, denoted toolMgoal
* compute the 6D error using the SE(3) log tool_nu
* compute the 6D jacobian tool_Jtool
* compute the control law vq = pinv(J)*nu
* integrate the robot velocity vq during DT to get a new configuration q
* log the error by storing it in a list herr.

In [ ]:
%do_not_load tp3/generated/inverse_kinematics_6d_loop

The tool frame F_tool converges toward the gooal frame F_goal: the center and the axes are finally aligned. The trajectory of the tool center is not a straight line, as the frame F_tool follows a "straight" line, not in R^3 but in SE(3).
We can also plot the error (assuming that herr is a list of the 6D errors herr).

In [ ]:
# %load tp3/generated/inverse_kinematics_plot
plt.subplot(211)
plt.plot([ e[:3] for e in herr])
plt.xlabel('control cycle (iter)')
plt.ylabel('error (m)')
plt.subplot(212)
plt.plot([ e[3:] for e in herr])
plt.xlabel('control cycle (iter)')
plt.ylabel('error (rad)');


### Inverse kinematics for two tasks
We yet controlled the robot with a single task (either the 3d position or the 6d placement errors). Let's see how to take into account a second task. Let's first introduce a second task. 

#### Introducing a second task: control the gaze
The robot has an additional frame named F_gaze, attached to the head and located 40 cm in front of the cameras. The task will be to position (3d) the center of this frame on an object of interest (a red ball).

In [52]:
# %load tp3/generated/control_head_robot
robot = loadTiago(addGazeFrame=True)

/opt/openrobots/lib/python3.8/site-packages/pinocchio/shortcuts.py:45: UserWarning: You passed package dir(s) via argument geometry_model and provided package_dirs.
  geom_model = pin.buildGeomFromUrdf(model, filename, geometry_type, package_dirs)


In [53]:
# %load tp3/generated/control_head_gaze
IDX_GAZE = robot.model.getFrameId('framegaze')

# Add a small ball as a visual target to be reached by the robot
ball = np.array([ 1.2,0.5,1.1 ])
viz.addSphere('ball', .05, [ .8,.1,.5, .8] )
viz.applyConfiguration('ball', list(ball)+[0,0,0,1])


In [54]:
viz.display(q0)
viz.viewer.jupyter_cell()

NameError: name 'q0' is not defined

Controlling this point can be done by achieving a simple variation of the control law for positioning (3d) the robot tool.

In [ ]:
%load tp3/generated/control_head_gaze_loop

q = q0.copy()
herr = [] # Log the value of the error between gaze and ball.
# Loop on an inverse kinematics for 200 iterations.
for i in range(200):  # Integrate over 2 second of robot life

    # Run the algorithms that outputs values in robot.data
    pin.framesForwardKinematics(robot.model,robot.data,q)
    pin.computeJointJacobians(robot.model,robot.data,q)

    # Placement from world frame o to frame f oMgaze
    oMgaze = robot.data.oMf[IDX_GAZE]

    # 6D jacobian in local frame
    o_Jgaze3 = pin.computeFrameJacobian(robot.model, robot.data, q, IDX_GAZE,pin.LOCAL_WORLD_ALIGNED)[:3,:]

    # vector from gaze to ball, in world frame
    o_GazeBall = oMgaze.translation-ball
    
    vq = -pinv(o_Jgaze3) @ o_GazeBall

    q = pin.integrate(robot.model,q, vq * DT)
    viz.display(q)
    time.sleep(1e-3)

    herr.append(o_GazeBall) 


#### Performing 2 tasks

We now have two tasks (e1,J1) controlling the tool and (e2,J2) controlling the gaze. 

The null space projector of +J1+ can be computed using the pseudoinverse.
Following the control law performing task 1 and task 2 in the null space of task 1 is:
$$vq_1 = J_1^+ v_1^*$$
$$P_1 = I_9 - J_1^+ J_1$$
$$vq_2 = vq_1 + (J_2 P_1)^+ ( v_2^* - J_2 vq_1)$$

You can now implement a control law solving the two tasks, i.e positioning the tool while controlling the gaze.

In [55]:
%do_not_load tp3/generated/control_head_multi

A third task can be implemented as well by computing the null space of the two first tasks

In [ ]:
Pgaze = Ptool - pinv(o_Jgaze3 @ Ptool) @ o_Jgaze3 @ Ptool

## Homework

Load an extra cube in the viewer to figure a table. First control the robot hand to reach an arbitrary point on the table (don't mind for the collision). Then implement a control law to control three tasks:
* the tool frame should be kept on the table (i.e. only the z component of the error matter, select only the 3rd row of the matrix).
* the gaze should be control to reach the position of a ball object positionned on the table.
* the center of the basis frame should reach a given target on the floor. For this task, only the x- and y- component of the task matter, select only the 2 first rows.
